In [70]:
pip install galois

In [71]:
# Needed libraries
import secrets
import math
import numpy as np
import hashlib
import galois

# Generate private seed

In [72]:
#Generate Private Seed
def generate_private_seed():
  return secrets.token_bytes(32)

# SHAKE A.K.A function H (Squeeze public_seed and T)

## SHAKE using hashlib

In [73]:
def create_private_sponge(private_seed, v:int, m:int, lvl:int):
  #Do the SHAKE
  if(lvl==1):
    h_shake = hashlib.shake_128(private_seed)
  else:
    h_shake = hashlib.shake_256(private_seed)
  #private sponge
  return h_shake.digest(32+math.ceil(m/8)*v)

#Extract public seed

In [74]:
def get_public_seed(private_sponge):
  return private_sponge[:32]

#Extract T

In [75]:
def int8_to_bits(intByte:int)->list:
  """Get the first 8 bits of an integer as an array.

  The function takes one integer byte, extracts it's bits
  and returns an array containing the bits starting from
  the most significant bit.

  Parameters
  ----------
  intByte : int
    The byte, labeled as an int by python.

  Returns
  -------
  list
      An array with the first 8 bits of the integer
      starting from the most significant one.
  """
  bits_data = bin(intByte)[2:]
  return [int(i) for i in bits_data.zfill(8 * ((len(bits_data) + 7) // 8))]

In [76]:
def tMatrixRowGen(byteString:bytes,neededBits:int)->list:
  """Generate one row of the T matrix.

  The function takes one byte-string and extracts the specified
  ammount of bits (neededBits) to fill one row. When the ammount
  of bits doesn't fit completely in a byte, the most significant
  bits of the last byte are ignored.

  Parameters
  ----------
  byteString : bytes
    A byte-string containing the bits used to fill the row.
  neededBits : int
    The ammount of bits a row needs to have.

  Returns
  -------
  list
      An array representing one row of the T matrix
  """
  row = []
  while(neededBits>8):
    row+=int8_to_bits(byteString[0])
    byteString = byteString[1:]
    neededBits = neededBits-8
  if(neededBits>0):
    row+=int8_to_bits(byteString[0])[-neededBits:]
  return row

def tMatrixGenerator(byteString:bytes,v:int,m:int)->np.ndarray:
  """Generate the T matrix.

  The function takes one byte-string and generates v rows
  each one with m bits extracted sequentially from a byteString
  using the tMatrixRowGen() function.

  Parameters
  ----------
  byteString : bytes
    A byte-string containing the bits used to fill the matrix.
  v : int
    The ammount rows of the matrix.
  m : int
    The ammount columns of the matrix.

  Returns
  -------
  list
      An array representing the T matrix
  """
  mdivided8 =math.ceil(m/8)
  matrix = []
  for i in range(v):
    matrix.append(tMatrixRowGen(byteString,m))
    byteString = byteString[mdivided8:]
  return np.array(matrix)

# SHAKE (squeeze public map)

In [77]:
def subMatrixGenerator(byteString:bytes)->np.ndarray:
  """Generate a part of the L or Q1 matrix.

  The function takes one byte-string and returns one 16 row part
  of the L or Q1 matrix. It must be noted that the rows are generated as columns
  at first and then are transposed.

  Parameters
  ----------
  byteString : bytes
    A byte-string containing the bits used to fill the matrix.

  Returns
  -------
  np.ndarray
      A matrix
  """

  matrix = [int8_to_bits(byteString[i*2]) + int8_to_bits(byteString[i*2+1]) for i in range(len(byteString)//2)]

  return np.array(matrix).T

In [78]:
def G(n, lvl, public_seed, g_equation, mdivided16):
  #first g function iteration
  if(lvl==1):
    g_shake = hashlib.shake_128(public_seed+bytes.fromhex('00'))
  else:
    g_shake = hashlib.shake_256(public_seed+bytes.fromhex('00'))
  public_sponge = g_shake.digest(g_equation)
  C_base=int8_to_bits(public_sponge[0])+int8_to_bits(public_sponge[1])
  public_sponge = public_sponge[2:]
  # L and Q1 have more than one dimension, so they are initilized
  # differently
  L_base = subMatrixGenerator(public_sponge[:2*n])
  public_sponge = public_sponge[2*n:]
  Q1_base = subMatrixGenerator(public_sponge)

  #following g function iterations
  if(lvl==1):
    for i in range(1, mdivided16):
      g_shake = hashlib.shake_128(public_seed+bytes.fromhex(f'0{i}'))
      public_sponge = g_shake.digest(g_equation)
      C_base+=int8_to_bits(public_sponge[0])+int8_to_bits(public_sponge[1])
      public_sponge = public_sponge[2:]
      L_base = np.concatenate((L_base,subMatrixGenerator(public_sponge[:2*n])))
      public_sponge = public_sponge[2*n:]
      Q1_base = np.concatenate((Q1_base,subMatrixGenerator(public_sponge)))
  else:
    for i in range(1, mdivided16):
      g_shake = hashlib.shake_256(public_seed+bytes.fromhex(f'0{i}'))
      public_sponge = g_shake.digest(g_equation)
      C_base+=int8_to_bits(public_sponge[0])+int8_to_bits(public_sponge[1])
      public_sponge = public_sponge[2:]
      L_base = np.concatenate((L_base,subMatrixGenerator(public_sponge[:2*n])))
      public_sponge = public_sponge[2*n:]
      Q1_base = np.concatenate((Q1_base,subMatrixGenerator(public_sponge)))

  return C_base,L_base,Q1_base

# find $$Q_2$$

In [79]:
def findPk1(v,m, k:int,q1:np.ndarray)->np.ndarray:
  """An implementation of the findPk1 algorithm of the LUOV specification.

  Parameters
  ----------
  k : int
    An integer between 1 and m.
  q1 : np.ndarray
    First part of Macaulay matrix of the quadratic part of P.

  Returns
  -------
  np.ndarray
      The v-by-v matrix representing the part of pk that is
      quadratic in the vinegar variables.
  """
  Pk1 = np.zeros([v,v],dtype=np.int8)
  column = 0
  for i in range(v):
    for j in range(i,v):
      Pk1[i,j]= q1[k,column]
      column+=1
    column+=m
  return Pk1

In [80]:
def findPk2(v, m, k:int,q1:np.ndarray)->np.ndarray:
  """An implementation of the findPk2 algorithm of the LUOV specification.

  Parameters
  ----------
  k : int
    An integer between 1 and m.
  q1 : np.ndarray
    First part of Macaulay matrix of the quadratic part of P.

  Returns
  -------
  np.ndarray
      The v-by-m matrix representing the part of pk that is bilinear in the
      vinegar variables and the oil variables.
  """
  Pk2 = np.zeros([v,m],dtype=np.int8)
  column = 0
  for i in range(v):
    column += v-i
    for j in range(m):
      Pk2[i,j]= q1[k,column]
      column+=1
  return Pk2

In [81]:
def findQ2(v, m, Q1:np.ndarray,T:np.ndarray)->galois.GF2:
  """An implementation of the findQ2 algorithm of the LUOV specification.

  Parameters
  ----------
  q1 : np.ndarray
    First part of Macaulay matrix of the quadratic part of P.
  t : np.ndarray
    A v-by-m matrix.

  Returns
  -------
  np.ndarray
      The second part of Macaulay matrix for quadratic part of P.
  """
  # we use galois to prevent the results of the operations from
  # being out of the finite field
  GF = galois.GF(2)
  T = GF(T)

  Q2 = GF(np.zeros([m,int(m*(m+1)/2)],dtype=np.int8))
  for k in range(m):
    Pk1 = GF(findPk1(v, m, k,Q1))
    Pk2 = GF(findPk2(v, m, k,Q1))
    Pk3 = -np.transpose(T)@Pk1@T+np.transpose(T)@Pk2
    column = 0
    for i in range(m):
      Q2[k,column]= Pk3[i,i]
      column += 1
      for j in range(i+1,m):
        Q2[k,column] = Pk3[i,j] + Pk3[j,i]
        column+=1
  return Q2

In [82]:
def LUOV(v:int, m:int, lvl:int):
  private_seed = generate_private_seed()
  private_sponge = create_private_sponge(private_seed,v,m,lvl)
  public_seed = get_public_seed(private_sponge)
  T_base = private_sponge[32:]
  T = tMatrixGenerator(T_base,v,m)
  n = m+v
  g_equation = 2+2*n+v*(v+1)+2*v*m
  mdivided16 = math.ceil(m/16)
  C_base, L_base, Q1_base = G(n, lvl, public_seed, g_equation, mdivided16)
  #obtaining C: we take the m bits (starting from the last generated bit) we need fromthe total of bits generated
  C = C_base[-m:]
  C = np.array(C)
  #obtaining L: We discard the upper rows to get the specified dimensions
  L = L_base[-m:]
  #obtaining Q: We discard the upper row to get the specified dimensions
  Q1=Q1_base[-m:]
  Q2 = findQ2(v, m, Q1,T)

  #GENERATE PUBLIC KEY

  # The first 32 bytes of the public key come from the public seed
  public_key = public_seed

  # The columns of Q2 are concatenated to get a sequence of bits for the public key
  Q2_bits = Q2.flatten('F')
  remaining_bits = len(Q2_bits)
  Q2_bytes = b''
  while(remaining_bits>8):
    byte = ''
    for i in Q2_bits[:8]:
      # Reverse the order of bits
      byte = str(Q2_bits[i]) + byte
    # Turn the bits into a byte and concatenate
    Q2_bytes+=int(byte,2).to_bytes(1,byteorder='big')
    remaining_bits -= 8
    Q2_bits = Q2_bits[8:]

  # If there are remaining bits, we add them to the last byte
  if (remaining_bits>0):
    last_byte = ''
    for bit in Q2_bits:
      # We reverse the order of
      last_byte = str(bit) + last_byte

  # Add the last byte
  Q2_bytes+=int(last_byte,2).to_bytes(1,byteorder='big')
  # Add the bytes to the public key
  public_key += Q2_bytes
  print(private_seed)
  print(public_key)
  print(len(public_key))

# Generate the public key

In [83]:
LUOV(197, 57, 1)

b"\xaa?}\x8e'a\xb8\xa7\xba\xa0\xc7\xdd \x9a,\xf2^m[m\x81W-\x1f\x9f\x05 \xfc:\xdf\xfe)"
b'lz\xc2\x0bM\xdf@[\xceu\xf6\xee(\xc2i.\xd6B\x94\x92jv\xce\x7f\xd4\xcfh\xe7b\x89>y&\x00\xfaz\xff~\xda\x12>\xb2\x00\x00\xff\xff\xffF\xff\x00\xb2\xff\xff\xff\xffj\xb6\xca\xff\xa2\x82~\xff*\x00\x00\x00\x1e\x02\xce\xff\x00^\xe6\xf6b\xff\x00\x9a\x00\xf2\xd2\xb2\x00\x00\xffR\xa2\xff\x00\x00*\x00\xff\x00\xff\xff\xb6\x00\x00\xae\xc2\xb2\x00\xe6\x00\xff\xff\xffvZ\xff\x00\x02\x00\x00Fr\xaa\x00b\xff\xff\x00\xffr\xfe\x96\xf6\xff\x00\xfe\x00\xbe\x00\xff\x00\xffF\xff\xfa\xd6\x92fn\x00\xff\x00\x00\xae\xf2\xb2&b\x00\xff^\x00.\x8e\n\xff\x00\xff\xffb\xff\x00\xb2\xff\xff^^\xc6\x00\x00\xee.\x00\xffRj\xff\xd2v\x00\xff\xff\x00\x00\xff\xffnv\x00\xff\x00\x9aR\x00\xba\x862V\xff\xff\xbe\x0e\x00\xff\x1e\xca\x92\x00\xc2\x00\x00f\x002\xff\xff\xff\x00\n\x00\xb2\x00\xa6\xff\xff\xff\x00\x00v\x00\x00\xb62\xe2\x00\xca\x00\xff\x1e\xff\xff2\x00\xd2\x8a\x00F\x00\xff\xffJ\xff\xbe\xff\xd6\xff\xff\xbe\x00\x00*\xb2j\xff:\xb2\xc2\x1e\xff6\xe

In [ ]:
LUOV(283, 83, 3)

In [ ]:
LUOV(374,110,5)